In [1]:
import pandas as pd
import re, os

In [ ]:
def describe_bug_files(PATH, bug_type):
    df = pd.read_csv(os.path.join(PATH, "{}_bug.csv".format(bug_type)) )
    print(
    df["releaseNo"].value_counts()
    )
    
    return

In [10]:
describe_bug_files(r"J:\EECS6444\Minke\Merge_Tables_For_Bugs\Bug_Result\org.eclipse.jdt.core", "pre")
describe_bug_files(r"J:\EECS6444\Minke\Merge_Tables_For_Bugs\Bug_Result\org.eclipse.jdt.core", "post")

R3_1      360
R3_0      353
R3_0_1    330
R3_1_1    264
R3_2      166
R3_3      163
R3_2_1    124
R3_3_1    111
R3_2_2    109
R3_5       92
R3_4_1     91
R3_1_2     88
R3_5_1     80
R3_4       64
R3_3_2     40
R3_4_2     40
R2_1_2     31
R2_1_1     29
R3_5_2     28
R2_0       23
R2_1_3     21
R2_1        5
R3_0_2      2
Name: releaseNo, dtype: int64
R3_3_1    14
R3_1_2    14
R3_0      13
R3_5_2    12
R3_2       9
R3_3       9
R3_1_1     8
R3_4_2     8
R3_3_2     6
R3_5       5
R3_2_2     5
R2_1       5
R3_4       5
R3_1       5
R3_5_1     4
R2_1_3     3
R3_2_1     3
R2_1_1     2
R3_4_1     2
R3_0_1     1
Name: releaseNo, dtype: int64


In [11]:
describe_bug_files(r"J:\EECS6444\Minke\Merge_Tables_For_Bugs\Bug_Result\org.eclipse.platform", "pre")
describe_bug_files(r"J:\EECS6444\Minke\Merge_Tables_For_Bugs\Bug_Result\org.eclipse.platform", "post")

R3_1_2    3
R3_1_1    2
R3_2      2
R3_2_1    2
R3_3_1    1
R2_1      1
R3_1      1
R2_1_1    1
R3_0_1    1
R2_0      1
Name: releaseNo, dtype: int64
R3_1      2
R3_1_1    1
Name: releaseNo, dtype: int64


In [1]:
import pandas as pd
import re

def change_versions(x):
    if x[-2:] == ".0":
        return "R{}".format( "_".join(   x[0:-2].split(".")) )
    else:
        return "R{}".format( "_".join(   x.split(".")) )

def merge_tables(TAB_PATH_1, TAB_PATH_2, TAB_PATH_3, Bug_List, how = "outer"):
    df1 = pd.read_csv(TAB_PATH_1)
    df1["date"] = df1["date"].map(lambda x: x[0:10].replace(".", "-"))  #pd.to_datetime(, format="%Y.%m.%d")
    df2 = pd.read_csv(TAB_PATH_2)
    df3 = pd.read_csv(TAB_PATH_3)
    df4 = pd.read_csv(Bug_List)
    ####
    df1.rename(columns={"fileVersion": "commitNo"}, inplace= True)
    df3.rename(columns={"Version": "releaseNo"}, inplace=True)
    df3["releaseNo"] = df3["releaseNo"].map(lambda x: change_versions(x))
    df4 = df4[["Bug ID", "Changed"]]
    df4["Changed"] = df4["Changed"].map(lambda x: x[0:10])
    df4.rename(columns={"Bug ID": "bugID"}, inplace=True)
    ####
    _ = pd.merge(df1,df2,on=['file','commitNo'],how= how)
    _1 = pd.merge(_, df3, on = ["releaseNo"], how = how)
    df_merge = pd.merge(_1, df4, on = ["bugID"], how = how)
    df_merge["date"] = pd.to_datetime(df_merge["date"], format = '%Y-%m-%d')
    df_merge["Date"] = pd.to_datetime(df_merge["Date"], format = '%Y-%m-%d')
    df_merge["Changed"] = pd.to_datetime(df_merge["Changed"], format = '%Y-%m-%d')
    ##
    df_merge["Changed-Date"] = df_merge["Changed"] - df_merge["Date"]
    ## 
    pre_bug = df_merge[(df_merge['Changed-Date'].dt.days > -180) & (df_merge['Changed-Date'].dt.days <= 0)]
    post_bug = df_merge[(df_merge['Changed-Date'].dt.days > 0) & (df_merge['Changed-Date'].dt.days <= 180)]
    return df_merge, pre_bug, post_bug

In [2]:
df, pre_bug, post_bug = merge_tables(
    r"org.eclipse.platform/org.eclipse.platform_no_pattern.csv", 
    r"org.eclipse.platform/org.eclipse.platform_release_commit.csv", 
    r"release-info.csv", 
    r"Platform_Bugs.csv",
    "inner"
)
pre_bug.to_csv("platform_pre_bug.csv", index=False)
post_bug.to_csv("platform_post_bug.csv", index=False)

In [3]:
df, pre_bug, post_bug = merge_tables(
    r"org.eclipse.jdt.core/org.eclipse.jdt.core_no_pattern.csv", 
    r"org.eclipse.jdt.core/org.eclipse.jdt.core_release_commit.csv", 
    r"release-info.csv",
    r"JDT_Bugs.csv",
    "inner"
)
pre_bug.to_csv("jdt.core_pre_bug.csv", index=False)
post_bug.to_csv("jdt.core_post_bug.csv", index=False)